In [ ]:
from pathlib import Path
import datetime
import shutil
import gzip

import matplotlib.pyplot as plt
import polars as pl
import numpy as np

import stock
import crypto
import data_fetcher

In [ ]:
# monthly -> dailyに変換
# とりあえずbtcusdt
symbol = "BTCUSDT"
monthly_csv_dir = Path("../../data_fetcher/data/binance/tick/monthly")
output_root_dir = Path("../../data_fetcher/data/binance/tick/")

for csv_path in sorted(monthly_csv_dir.rglob(f"{symbol}*-trades*.csv.gz")):
    try:
        df = pl.read_csv(csv_path)
    except:
        print(csv_path)
    split_dfs = df.with_columns(
        pl.from_epoch(pl.col("time"), "ms").cast(pl.Date).alias("date")
    ).partition_by("date")

    for df in split_dfs:
        date = df[0]["date"][0].strftime("%Y%m%d")
        output_path = output_root_dir / date / f"{symbol}-trades-{date}.csv.gz"
        output_path.parent.mkdir(exist_ok=True, parents=False)
        if output_path.exists():
            continue
        with gzip.open(output_path, "wb") as f:
            df.write_csv(f)
        print(output_path)    

In [ ]:
# base strategy

# boxの判定 : MAEの傾きが小さい

In [ ]:
start_date = datetime.datetime(2024, 12, 1)
end_date = datetime.datetime(2024, 12, 31)

# fetcher = data_fetcher.gmo.GMOFethcer()
# gmo_df = fetcher.fetch_ticker("BTC_JPY", start_date=start_date, end_date=end_date)

In [ ]:
df = pl.read_csv("/home/kitamura/work/data_fetcher/data/binance/tick/202412/BTCUSDT-trades-2024-12.csv.gz")

In [ ]:
fetcher = data_fetcher.binance.BinanceFetcher()
bin_df = fetcher.fetch_ticker("BTCUSDT", start_date=start_date, end_date=end_date)

In [ ]:
df

In [ ]:
df = df.select(
    pl.lit("BTCUSDT").alias("symbol"),
    pl.when(pl.col("isBuyerMaker"))
    .then(pl.lit("BUY"))
    .otherwise(pl.lit("SELL"))
    .alias("side"),
    pl.col("price"),
    pl.col("quantity").alias("size"),
    pl.from_epoch(pl.col("Timestamp"), "ms").alias("datetime")
)


In [ ]:
df

In [ ]:
start = datetime.datetime(2024, 12, 2, 2, 0, 0)
end = start + datetime.timedelta(minutes=60)
gmo_target = gmo_df.filter(pl.col("datetime").is_between(start, end))
bin_target = bin_df.filter(pl.col("datetime").is_between(start, end))

In [ ]:
plt.plot(gmo_target["datetime"], gmo_target["price"])
plt.plot(bin_target["datetime"], bin_target["price"])

In [ ]:
plt.plot(df["price"])

In [ ]:
plt.figure(figsize=(20, 5))
si, ei = 250, 350
plt.plot(df["datetime"][si:ei], df["price"][si:ei], marker="o", markersize=7)
plt.grid()

In [ ]:
plt.plot(df["price"])